In [41]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [42]:
filepath = tf.keras.utils.get_file('shakespeare.txt', './data/shakespeare.txt')

In [43]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [44]:
text = text[100000:1000000]

In [45]:
characters = sorted(set(text))

In [46]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [97]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [98]:
sentences = []
next_characters = []

In [99]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i + SEQ_LENGTH])
  next_characters.append(text[i + SEQ_LENGTH])

In [100]:
X = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
Y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

In [101]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    X[i, t, char_to_index[character]] = 1
  Y[i, char_to_index[next_characters[i]]] = 1

In [106]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
csv_logger = tf.keras.callbacks.CSVLogger('./training_log/log.csv', separator=',', append=True)

model.fit(X, Y, batch_size=256, epochs=10, callbacks=[csv_logger])

Epoch 1/10
1172/1172 [==============================] - 74s 62ms/step - loss: 1.9404
Epoch 2/10
1172/1172 [==============================] - 73s 62ms/step - loss: 1.5849
Epoch 3/10
1172/1172 [==============================] - 73s 62ms/step - loss: 1.5010
Epoch 4/10
1172/1172 [==============================] - 73s 62ms/step - loss: 1.4564
Epoch 5/10
1172/1172 [==============================] - 80s 68ms/step - loss: 1.4289
Epoch 6/10
1172/1172 [==============================] - 74s 63ms/step - loss: 1.4067
Epoch 7/10
1172/1172 [==============================] - 74s 63ms/step - loss: 1.3906
Epoch 8/10
1172/1172 [==============================] - 74s 63ms/step - loss: 1.3782
Epoch 9/10
1172/1172 [==============================] - 74s 63ms/step - loss: 1.3680
Epoch 10/10
1172/1172 [==============================] - 74s 63ms/step - loss: 1.3577


In [107]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


In [108]:
model = tf.keras.models.load_model('textgenerator.model')

In [109]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [110]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    A = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence):
      A[0, t, char_to_index[character]] = 1
    
    predictions = model.predict(A, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character
  
  return generated

In [111]:
print('-----------------------------0.2-------------------------------')
print(generate_text(300, 0.2))
print('-----------------------------0.4-------------------------------')
print(generate_text(300, 0.4))
print('-----------------------------0.6-------------------------------')
print(generate_text(300, 0.6))
print('-----------------------------0.8-------------------------------')
print(generate_text(300, 0.8))
print('------------------------------1--------------------------------')
print(generate_text(300, 1))

-----------------------------0.2-------------------------------
ruth
to god, my king and my succeeding in the son.

king henry vi:
what shall be the state of the state, the sea
the death and the service of the sun the state,
to see the state and bear the sun the state,
and shall be the sun the state to hear thee,
and the state here and the son and the heart,
and i will be the state and the perion.

ki
-----------------------------0.4-------------------------------


john of gaunt:
will the king come, that the very wall the heaven
and the court is thy brother and the cause and pity,
which could was the since i will prove thee,
and let the king and sign to hear thee was
a liver the strike and the court is a wrong,
but the king of here, and shall not be the son.

duke of york:
how is your contraded to
-----------------------------0.6-------------------------------
wife and my poor children!
i pray thee, the edward stands to her none?

northumberland:
and they say the rest in the nothing t